In [87]:
import os
import pandas as pd
import json
from tqdm import tqdm
import requests
from IPython.display import display, Markdown
import ipywidgets as widgets
from prompt_techniques import PromptTechniques
from pathlib import Path
import re
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import Dataset
import torch
from rouge import Rouge
from transformers import logging as transformers_logging

In [1]:
!pip uninstall transformers -y

Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3


In [1]:
!pip install tokenizers==0.13.3 --no-deps  
!pip install transformers==4.31.0

  Using cached tokenizers-0.13.3.tar.gz (314 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build tokenizers


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [62 lines of output]
  C:\Users\adenk\AppData\Local\Temp\pip-build-env-g0bwij9z\overlay\Lib\site-packages\setuptools\dist.py:761: SetuptoolsDeprecationWarning: License classifiers are deprecated.
  !!
  
          ********************************************************************************
          Please consider removing the following classifiers in favor of a SPDX license expression:
  
          License :: OSI Approved :: Apache Software License
  
          See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
          ********************************************************************************
  
  !!
    self._finalize_license_expression()
  running bdist_wheel
  running build
  running build_py
  creating build\lib.win-amd64-cpython-313\tokenizers
  copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-313\tokenizers
  creating build\lib.win-amd64-cpyth

  Obtaining dependency information for transformers==4.31.0 from https://files.pythonhosted.org/packages/21/02/ae8e595f45b6c8edee07913892b3b41f5f5f273962ad98851dc6a564bbb9/transformers-4.31.0-py3-none-any.whl.metadata
  Using cached transformers-4.31.0-py3-none-any.whl.metadata (116 kB)
  Using cached tokenizers-0.13.3.tar.gz (314 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
Failed to build tokenizers


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [62 lines of output]
  C:\Users\adenk\AppData\Local\Temp\pip-build-env-w5mixx1w\overlay\Lib\site-packages\setuptools\dist.py:761: SetuptoolsDeprecationWarning: License classifiers are deprecated.
  !!
  
          ********************************************************************************
          Please consider removing the following classifiers in favor of a SPDX license expression:
  
          License :: OSI Approved :: Apache Software License
  
          See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
          ********************************************************************************
  
  !!
    self._finalize_license_expression()
  running bdist_wheel
  running build
  running build_py
  creating build\lib.win-amd64-cpython-313\tokenizers
  copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-313\tokenizers
  creating build\lib.win-amd64-cpyth

In [99]:
!pip install --force-reinstall accelerate>=0.26.0

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\adenk\\PycharmProjects\\code-to-doc\\.venv\\Lib\\site-packages\\~afetensors\\_safetensors_rust.pyd'
Check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [97]:
!pip install torch transformers[torch] accelerate>=0.26.0


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
from transformers import __version__ as transformers_version
from accelerate import __version__ as accelerate_version

print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers_version}")
print(f"Accelerate: {accelerate_version}")

ModuleNotFoundError: No module named 'transformers'

In [2]:
# Configuration Cell - Set these variables before running
DATA_DIR = '../dataset'  # Directory containing your datasets
OUTPUT_DIR = '../output'  # Directory to save results
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:

# %%
# Helper functions
def load_dataset(file_path):
    """Load dataset from CSV file"""
    try:
        df = pd.read_csv(file_path)
        print(f"Dataset loaded successfully. Shape: {df.shape}")
        return df
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

def save_results(df, filename):
    """Save results to CSV"""
    output_path = os.path.join(OUTPUT_DIR, filename)
    df.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

# %%
# Model and Technique Selection
model_options = ['llama3.2:1b', 'qwen2.5-coder:0.5b', 'deepseek-r1:1.5b']
technique_options = {
    '1': 'Zero-shot',
    '2': 'Few-shot', 
    '3': 'Chain-of-thought',
    '4': 'Structured',
    '5': 'One-shot'
}

# Create interactive widgets
# Reverse mapping for display names to keys
technique_keys = {v: k for k, v in technique_options.items()}

# Create interactive widgets
model_dropdown = widgets.Dropdown(options=model_options, description='Model:')
technique_dropdown = widgets.Dropdown(options=list(technique_options.values()), description='Technique:')
dataset_files = [f for f in os.listdir(DATA_DIR) if f.endswith('.csv')]
dataset_dropdown = widgets.Dropdown(options=dataset_files, description='Dataset:')

display(model_dropdown, technique_dropdown, dataset_dropdown)



Dropdown(description='Model:', options=('llama3.2:1b', 'qwen2.5-coder:0.5b', 'deepseek-r1:1.5b'), value='llama…

Dropdown(description='Technique:', options=('Zero-shot', 'Few-shot', 'Chain-of-thought', 'Structured', 'One-sh…

Dropdown(description='Dataset:', options=('ctuning_ck.csv', 'data.csv', 'HHammond_PrettyPandas.csv', 'michaelp…

In [31]:
# Main Processing Cell
def run_generator(model, technique_name, dataset_file):
    """Run the documentation generator with selected parameters"""
    # Convert technique name back to key
    technique = technique_keys[technique_name]
    
    # Load dataset
    df = load_dataset(os.path.join(DATA_DIR, dataset_file))
    if df is None:
        return None
    
    # Initialize prompt techniques
    pt = PromptTechniques(model)
    
    # Prepare examples (from your main.py)
    examples = [
        {"input": "def add(a, b): return a + b", "output": "Parameters: a first number, b first number Returns: sum of two numbers"}, 
        {"input": "def greet(name): print(f\"Hello, {name}\")", "output": "Greets a user by name."},
        # ... (include all your examples from main.py)
    ]
    
    results = []
    
    # Process each function in the dataset
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Generating docs"):
        func = row['func_code_string']
        docstring = row.get('func_documentation_string', '')
        
        # Apply selected technique
        generated = None
        try:
            if technique == '1':
                generated = pt.zero_shot_prompting(func, 1)
            elif technique == '2':
                generated = pt.few_shot_prompting(func, examples, 1)
            elif technique == '3':
                generated = pt.chain_of_thought_prompting(func, 1)
            elif technique == '4':
                generated = pt.structured_prompting(func, 1)
            elif technique == '5':
                generated = pt.one_shot_prompting(func, examples, 1)
        except Exception as e:
            print(f"Error generating docs for {row.get('func_name', '')}: {str(e)}")
            generated = None
        
        # Store results with proper key handling
        result = {
            'filename': row.get('repository_name', ''),
            'function_name': row.get('func_name', ''),
            'input_code': func,
            'model': model,
            'technique': technique_name,
            'original_doc': docstring,
            'prompt_used': pt.prompt,
            'generated_doc': ''  # Default empty string
        }
        
        if generated:
            if technique in ['1', '4']:  # Zero-shot and Structured use 'output'
                result['generated_doc'] = generated[0].get('output', '')
            elif technique in ['2', '3', '5']:  # Few-shot, Chain-of-thought, One-shot use 'model_output'
                result['generated_doc'] = generated[0].get('model_output', '')
        
        results.append(result)
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# %%
# Execute with selected parameters
selected_model = model_dropdown.value
selected_technique_name = technique_dropdown.value  # Get the display name
selected_dataset = dataset_dropdown.value

results_df = run_generator(selected_model, selected_technique_name, selected_dataset)


Dataset loaded successfully. Shape: (5, 11)


Generating docs: 100%|██████████| 5/5 [03:14<00:00, 38.86s/it]


In [ ]:
# Display 
if results_df is not None:
    # Display sample results
    display(Markdown("### Sample Generated Documentation"))
    sample = results_df.sample(min(3, len(results_df)))
    for _, row in sample.iterrows():
        display(Markdown(f"**Function:** {row['function_name']}"))
        display(Markdown(f"**Original Code:**\n```python\n{row['input_code']}\n```"))
        display(Markdown(f"**Generated Docstring:**\n```\n{row['generated_doc']}\n```"))
        display(Markdown("---"))
    
    # Show full DataFrame
    display(Markdown("### Full Results DataFrame"))
    display(results_df.head())

In [32]:
results_df.head()

,filename,function_name,input_code,model,technique,original_doc,prompt_used,generated_doc
0,tadeck/onetimepass,_is_possible_token,"def _is_possible_token(token, token_length=6):...",deepseek-r1:1.5b,One-shot,Determines if given value is acceptable as a t...,"Here is one example:\nFunction:\ndef add(a, b)...","<think>\nAlright, so I'm looking at this probl..."
1,tadeck/onetimepass,get_hotp,"def get_hotp(\r\n secret,\r\n in...",deepseek-r1:1.5b,One-shot,Get HMAC-based one-time password on the basis ...,"Here is one example:\nFunction:\ndef add(a, b)...","<think>\nAlright, I need to write a Python doc..."
2,tadeck/onetimepass,get_totp,"def get_totp(\r\n secret,\r\n as...",deepseek-r1:1.5b,One-shot,Get time-based one-time password on the basis ...,"Here is one example:\nFunction:\ndef add(a, b)...","<think>\nAlright, let's tackle this problem st..."
3,tadeck/onetimepass,valid_hotp,"def valid_hotp(\r\n token,\r\n s...",deepseek-r1:1.5b,One-shot,Check if given token is valid for given secret...,"Here is one example:\nFunction:\ndef add(a, b)...","<think>\nAlright, I need to write a docstring ..."
4,tadeck/onetimepass,valid_totp,"def valid_totp(\r\n token,\r\n s...",deepseek-r1:1.5b,One-shot,Check if given token is valid time-based one-t...,"Here is one example:\nFunction:\ndef add(a, b)...","<think>\nAlright, let's tackle this problem st..."


llama

In [15]:
results_df.to_csv('../output/llama_zero.csv', index=False)

In [25]:
results_df.to_csv('../output/llama_few.csv', index=False)

In [29]:
results_df.to_csv('../output/llama_chain.csv', index=False)

In [31]:
results_df.to_csv('../output/llama_structured.csv', index=False)

In [34]:
results_df.to_csv('../output/llama_oneshot.csv', index=False)

qwen

In [6]:
results_df.to_csv('../output/qwen_zero.csv', index = False)

In [9]:
results_df.to_csv('../output/qwen_few.csv', index = False)

In [12]:
results_df.to_csv('../output/qwen_chain.csv', index = False)

In [15]:
results_df.to_csv('../output/qwen_structured.csv', index = False)

In [18]:
results_df.to_csv('../output/qwen_oneshot.csv', index = False)

deepseek

In [21]:
results_df.to_csv('../output/deep_zero.csv', index = False)

In [24]:
results_df.to_csv('../output/deep_few.csv', index = False)

In [27]:
results_df.to_csv('../output/deep_chain.csv', index = False)

In [30]:
results_df.to_csv('../output/deep_structured.csv', index = False)

In [33]:
results_df.to_csv('../output/deep_oneshot.csv', index = False)

In [36]:
def extract_docstring(text):
    """
    Extract just the docstring content from model output, handling different formats.
    Removes code fences, language markers, and other non-docstring content.
    """
    if not isinstance(text, str):
        return ""
    
    # Pattern for Google-style docstrings (triple quotes with optional whitespace)
    docstring_pattern = r'\"\"\"(.*?)\"\"\"|\'\'\'(.*?)\'\'\''
    
    # Try to find docstring pattern first
    matches = re.findall(docstring_pattern, text, re.DOTALL)
    if matches:
        # Join all matches (handles cases with both single and double quotes)
        combined = " ".join([" ".join(match) for match in matches])
        return combined.strip()
    
    # If no docstring pattern found, look for markdown code blocks
    code_block_pattern = r'```(?:python)?\n(.*?)\n```'
    code_matches = re.findall(code_block_pattern, text, re.DOTALL)
    if code_matches:
        return code_matches[0].strip()
    
    # Fallback - return first 3 lines if no clear pattern found
    return "\n".join(text.split("\n")[:3]).strip()

def process_output_file(file_path):
    """Process a single output file to extract clean docstrings"""
    try:
        df = pd.read_csv(file_path)
        
        # Create new column for cleaned docstrings
        df['clean_docstring'] = df['generated_doc'].apply(extract_docstring)
        
        # Save back to same file with additional column
        df.to_csv(file_path, index=False)
        print(f"Processed and saved: {file_path}")
        return df
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return None

In [37]:
process_output_file('../output/llama_zero.csv')

Processed and saved: ../output/llama_zero.csv


,filename,function_name,input_code,model,technique,generated_doc,original_doc,prompt_used,clean_docstring
0,tadeck/onetimepass,_is_possible_token,"def _is_possible_token(token, token_length=6):...",llama3.2:1b,Zero-shot,```\ndef _is_possible_token(\n token: int |...,Determines if given value is acceptable as a t...,"As a senior Python engineer, write a professio...",Determines if given value is acceptable as a t...
1,tadeck/onetimepass,get_hotp,"def get_hotp(\r\n secret,\r\n in...",llama3.2:1b,Zero-shot,"```\ndef get_hotp(\n secret: str or bytes,\...",Get HMAC-based one-time password on the basis ...,"As a senior Python engineer, write a professio...",Get HMAC-based one-time password on the basis ...
2,tadeck/onetimepass,get_totp,"def get_totp(\r\n secret,\r\n as...",llama3.2:1b,Zero-shot,"```\ndef get_totp(\n secret,\n as_string...",Get time-based one-time password on the basis ...,"As a senior Python engineer, write a professio...",Generate a time-based one-time password (TOTP)...
3,tadeck/onetimepass,valid_hotp,"def valid_hotp(\r\n token,\r\n s...",llama3.2:1b,Zero-shot,"```\ndef valid_hotp(\n token: int or str, ...",Check if given token is valid for given secret...,"As a senior Python engineer, write a professio...",Check if given token is valid for given secret...
4,tadeck/onetimepass,valid_totp,"def valid_totp(\r\n token,\r\n s...",llama3.2:1b,Zero-shot,"```\n""""""\nChecks if a given time-based one-tim...",Check if given token is valid time-based one-t...,"As a senior Python engineer, write a professio...",Checks if a given time-based one-time password...


In [40]:
process_output_file('../output/llama_few.csv')
process_output_file('../output/llama_chain.csv')
process_output_file('../output/llama_structured.csv')
process_output_file('../output/llama_oneshot.csv')

Processed and saved: ../output/llama_few.csv
Processed and saved: ../output/llama_chain.csv
Processed and saved: ../output/llama_structured.csv
Processed and saved: ../output/llama_oneshot.csv


,filename,function_name,input_code,model,technique,original_doc,prompt_used,generated_doc,clean_docstring
0,tadeck/onetimepass,_is_possible_token,"def _is_possible_token(token, token_length=6):...",llama3.2:1b,One-shot,Determines if given value is acceptable as a t...,"Here is one example:\nFunction:\ndef add(a, b)...",Here is a possible docstring for the `_is_poss...,Determines if given value is acceptable as a t...
1,tadeck/onetimepass,get_hotp,"def get_hotp(\r\n secret,\r\n in...",llama3.2:1b,One-shot,Get HMAC-based one-time password on the basis ...,"Here is one example:\nFunction:\ndef add(a, b)...",Here is a possible docstring for the `get_hotp...,Get HMAC-based one-time password on the basis ...
2,tadeck/onetimepass,get_totp,"def get_totp(\r\n secret,\r\n as...",llama3.2:1b,One-shot,Get time-based one-time password on the basis ...,"Here is one example:\nFunction:\ndef add(a, b)...",Here is a possible docstring for the `get_totp...,Generate time-based one-time password (TOTP) b...
3,tadeck/onetimepass,valid_hotp,"def valid_hotp(\r\n token,\r\n s...",llama3.2:1b,One-shot,Check if given token is valid for given secret...,"Here is one example:\nFunction:\ndef add(a, b)...",Here is a possible docstring for the `valid_ho...,Check if given token is valid for given secret...
4,tadeck/onetimepass,valid_totp,"def valid_totp(\r\n token,\r\n s...",llama3.2:1b,One-shot,Check if given token is valid time-based one-t...,"Here is one example:\nFunction:\ndef add(a, b)...",Here is a docstring for the `valid_totp` funct...,Checks if given TOTP (Time-Based One-Time Pass...


In [41]:
process_output_file('../output/qwen_zero.csv')
process_output_file('../output/qwen_few.csv')
process_output_file('../output/qwen_chain.csv')
process_output_file('../output/qwen_structured.csv')
process_output_file('../output/qwen_oneshot.csv')

Processed and saved: ../output/qwen_zero.csv
Processed and saved: ../output/qwen_few.csv
Processed and saved: ../output/qwen_chain.csv
Processed and saved: ../output/qwen_structured.csv
Processed and saved: ../output/qwen_oneshot.csv


,filename,function_name,input_code,model,technique,original_doc,prompt_used,generated_doc,clean_docstring
0,tadeck/onetimepass,_is_possible_token,"def _is_possible_token(token, token_length=6):...",qwen2.5-coder:0.5b,One-shot,Determines if given value is acceptable as a t...,"Here is one example:\nFunction:\ndef add(a, b)...","```python\ndef _is_possible_token(token, token...",Determines if given value is acceptable as a t...
1,tadeck/onetimepass,get_hotp,"def get_hotp(\r\n secret,\r\n in...",qwen2.5-coder:0.5b,One-shot,Get HMAC-based one-time password on the basis ...,"Here is one example:\nFunction:\ndef add(a, b)...",Here's a complete docstring for the `get_hotp`...,Get HMAC-based one-time password on the basis ...
2,tadeck/onetimepass,get_totp,"def get_totp(\r\n secret,\r\n as...",qwen2.5-coder:0.5b,One-shot,Get time-based one-time password on the basis ...,"Here is one example:\nFunction:\ndef add(a, b)...",The following is a docstring for the given fun...,Get time-based one-time password on the basis ...
3,tadeck/onetimepass,valid_hotp,"def valid_hotp(\r\n token,\r\n s...",qwen2.5-coder:0.5b,One-shot,Check if given token is valid for given secret...,"Here is one example:\nFunction:\ndef add(a, b)...",Here is the docstring for the `valid_hotp` fun...,Check if given token is valid for given secret...
4,tadeck/onetimepass,valid_totp,"def valid_totp(\r\n token,\r\n s...",qwen2.5-coder:0.5b,One-shot,Check if given token is valid time-based one-t...,"Here is one example:\nFunction:\ndef add(a, b)...","```python\ndef valid_totp(\n token,\n ...",Check if given token is valid time-based one-t...


In [42]:
process_output_file('../output/deep_zero.csv')
process_output_file('../output/deep_few.csv')
process_output_file('../output/deep_chain.csv')
process_output_file('../output/deep_structured.csv')
process_output_file('../output/deep_oneshot.csv')

Processed and saved: ../output/deep_zero.csv
Processed and saved: ../output/deep_few.csv
Processed and saved: ../output/deep_chain.csv
Processed and saved: ../output/deep_structured.csv
Processed and saved: ../output/deep_oneshot.csv


,filename,function_name,input_code,model,technique,original_doc,prompt_used,generated_doc,clean_docstring
0,tadeck/onetimepass,_is_possible_token,"def _is_possible_token(token, token_length=6):...",deepseek-r1:1.5b,One-shot,Determines if given value is acceptable as a t...,"Here is one example:\nFunction:\ndef add(a, b)...","<think>\nAlright, so I'm looking at this probl...",Determines if given value is acceptable as a t...
1,tadeck/onetimepass,get_hotp,"def get_hotp(\r\n secret,\r\n in...",deepseek-r1:1.5b,One-shot,Get HMAC-based one-time password on the basis ...,"Here is one example:\nFunction:\ndef add(a, b)...","<think>\nAlright, I need to write a Python doc...",">>> get_hotp(b'MFRGGZDFMZTWQ2LK', intervals_no..."
2,tadeck/onetimepass,get_totp,"def get_totp(\r\n secret,\r\n as...",deepseek-r1:1.5b,One-shot,Get time-based one-time password on the basis ...,"Here is one example:\nFunction:\ndef add(a, b)...","<think>\nAlright, let's tackle this problem st...",Get time-based one-time password (OTTP) on the...
3,tadeck/onetimepass,valid_hotp,"def valid_hotp(\r\n token,\r\n s...",deepseek-r1:1.5b,One-shot,Check if given token is valid for given secret...,"Here is one example:\nFunction:\ndef add(a, b)...","<think>\nAlright, I need to write a docstring ...",>>> secret = b'MFRGGZDFMZTWQ2LK'\n>>> valid_ho...
4,tadeck/onetimepass,valid_totp,"def valid_totp(\r\n token,\r\n s...",deepseek-r1:1.5b,One-shot,Check if given token is valid time-based one-t...,"Here is one example:\nFunction:\ndef add(a, b)...","<think>\nAlright, let's tackle this problem st...",>>> secret = b'MFRGGZDFMZTWQ2LK'\n>>> token = ...


In [43]:
test = pd.read_csv('../output/qwen_structured.csv')

,filename,function_name,input_code,model,technique,original_doc,prompt_used,generated_doc,clean_docstring
0,tadeck/onetimepass,_is_possible_token,"def _is_possible_token(token, token_length=6):...",qwen2.5-coder:0.5b,Structured,Determines if given value is acceptable as a t...,You are a senior Python developer and technica...,"```python\n""""""Determines if given value is acc...",Determines if given value is acceptable as a t...
1,tadeck/onetimepass,get_hotp,"def get_hotp(\r\n secret,\r\n in...",qwen2.5-coder:0.5b,Structured,Get HMAC-based one-time password on the basis ...,You are a senior Python developer and technica...,"```python\ndef get_hotp(secret, intervals_no, ...",Get HMAC-based one-time password on the basis ...
2,tadeck/onetimepass,get_totp,"def get_totp(\r\n secret,\r\n as...",qwen2.5-coder:0.5b,Structured,Get time-based one-time password on the basis ...,You are a senior Python developer and technica...,"```python\ndef get_totp(secret, as_string=Fals...",Get time-based one-time password on the basis ...
3,tadeck/onetimepass,valid_hotp,"def valid_hotp(\r\n token,\r\n s...",qwen2.5-coder:0.5b,Structured,Check if given token is valid for given secret...,You are a senior Python developer and technica...,```,```
4,tadeck/onetimepass,valid_totp,"def valid_totp(\r\n token,\r\n s...",qwen2.5-coder:0.5b,Structured,Check if given token is valid time-based one-t...,You are a senior Python developer and technica...,```python\n# Function to validate TOTP for a g...,Check if given token is valid time-based one-t...


In [45]:
test[['function_name', 'original_doc', 'clean_docstring']]

,function_name,original_doc,clean_docstring
0,_is_possible_token,Determines if given value is acceptable as a t...,Determines if given value is acceptable as a t...
1,get_hotp,Get HMAC-based one-time password on the basis ...,Get HMAC-based one-time password on the basis ...
2,get_totp,Get time-based one-time password on the basis ...,Get time-based one-time password on the basis ...
3,valid_hotp,Check if given token is valid for given secret...,```
4,valid_totp,Check if given token is valid time-based one-t...,Check if given token is valid time-based one-t...


Training y testing y tuning

In [88]:
# Cell 2: Evaluation Function
def evaluate_docstrings(df):
    """
    Evaluate generated docstrings against original docstrings using multiple metrics.
    Returns a dictionary of scores.
    """
    # Load sentence transformer model for semantic similarity
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    results = {}
    
    # Filter out empty docstrings
    valid_pairs = df[(df['clean_docstring'].str.len() > 0) & 
                    (df['original_doc'].str.len() > 0)].copy()
    
    if len(valid_pairs) == 0:
        return {"error": "No valid docstring pairs to evaluate"}
    
    # Calculate exact match (strict comparison)
    valid_pairs['exact_match'] = valid_pairs.apply(
        lambda x: x['clean_docstring'].strip() == x['original_doc'].strip(), axis=1)
    results['exact_match_accuracy'] = valid_pairs['exact_match'].mean()
    
    # Calculate semantic similarity using cosine similarity
    original_embeddings = model.encode(valid_pairs['original_doc'].tolist())
    generated_embeddings = model.encode(valid_pairs['clean_docstring'].tolist())
    cosine_scores = util.cos_sim(original_embeddings, generated_embeddings)
    results['mean_cosine_similarity'] = np.mean(np.diag(cosine_scores))
    
    # Calculate ROUGE scores (for text similarity)
    rouge = Rouge()
    rouge_scores = rouge.get_scores(
        valid_pairs['clean_docstring'].tolist(),
        valid_pairs['original_doc'].tolist(),
        avg=True)
    results.update({
        'rouge-1': rouge_scores['rouge-1']['f'],
        'rouge-2': rouge_scores['rouge-2']['f'],
        'rouge-l': rouge_scores['rouge-l']['f']
    })
    
    return results

In [89]:
# Cell 3: Training Data Preparation
def prepare_training_data(df):
    """Prepare dataset for fine-tuning"""
    # Filter out empty samples
    df = df[(df['input_code'].str.len() > 0) & 
            (df['original_doc'].str.len() > 0)].copy()
    
    # Create dataset
    dataset = Dataset.from_pandas(df[['input_code', 'original_doc']])
    
    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained("t5-small")
    
    def preprocess_function(examples):
        inputs = ["Generate docstring: " + code for code in examples["input_code"]]
        targets = examples["original_doc"]
        model_inputs = tokenizer(inputs, max_length=512, truncation=True)
        
        # Setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=128, truncation=True)
        
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    
    tokenized_dataset = dataset.map(preprocess_function, batched=True)
    return tokenized_dataset, tokenizer

In [90]:
# Updated Cell 4: Model Training 
from accelerate import Accelerator

def train_model(df, model_name="t5-small", output_dir="../output"):
    """Fine-tune a model on docstring generation"""
    # Initialize accelerator
    accelerator = Accelerator()
    
    # Prepare data
    tokenized_dataset, tokenizer = prepare_training_data(df)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        eval_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        save_total_limit=3,
        report_to="none",
        logging_strategy="epoch",
        save_strategy="epoch",
        fp16=True if accelerator.mixed_precision == "fp16" else False  # Auto-detect mixed precision
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer,
    )
    
    # Accelerate preparation
    trainer.train()
    trainer.save_model(output_dir)
    return trainer, tokenizer

In [91]:
# Cell 5: Model Evaluation
def evaluate_all_models(output_dir):
    """Evaluate all model outputs in the directory"""
    results = []
    output_path = Path(output_dir)
    
    for file in output_path.glob('*.csv'):
        if file.is_file():
            df = pd.read_csv(file)
            model_name = file.stem.split('_')[0]
            technique = '_'.join(file.stem.split('_')[1:-1])
            
            # Evaluate
            metrics = evaluate_docstrings(df)
            
            # Add metadata
            metrics.update({
                'model': model_name,
                'technique': technique,
                'file': file.name,
                'num_samples': len(df)
            })
            
            results.append(metrics)
    
    return pd.DataFrame(results)

In [92]:
# Cell 6: Generate Predictions
def generate_predictions(trainer, tokenizer, df):
    """Generate predictions using trained model"""
    # Prepare dataset
    tokenized_dataset, _ = prepare_training_data(df)
    
    # Generate predictions
    predictions = trainer.predict(tokenized_dataset)
    
    # Decode predictions
    decoded_preds = tokenizer.batch_decode(
        predictions.predictions,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )
    
    # Add to DataFrame
    result_df = df.copy()
    result_df['model_prediction'] = decoded_preds[:len(result_df)]
    
    return result_df

In [95]:
# Cell 7: Execute Pipeline (Final Version)
import torch
from transformers import __version__ as transformers_version
from accelerate import __version__ as accelerate_version

# Reduce verbosity of transformers logging
transformers_logging.set_verbosity_error()

try:
    # 1. First check the output directory
    output_dir = Path('../output')
    csv_files = list(output_dir.glob('*.csv'))
    
    if not csv_files:
        raise ValueError(f"No CSV files found in {output_dir.absolute()}")
    
    print(f"Found {len(csv_files)} files to process")
    
    # 2. Evaluate all models
    print("\nEvaluating all models...")
    eval_results = evaluate_all_models('output/')
    eval_results.to_csv('evaluation_results.csv', index=False)
    display(eval_results)
    
    # 3. Combine all data for training
    print("\nLoading datasets for training...")
    all_data = []
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            if not df.empty:
                print(f"Loaded {file.name} ({len(df)} rows)")
                all_data.append(df)
        except Exception as e:
            print(f"Error loading {file.name}: {str(e)}")
            continue
    
    if not all_data:
        raise ValueError("No valid data loaded - cannot proceed with training")
    
    full_dataset = pd.concat(all_data, ignore_index=True)
    print(f"\nCombined dataset contains {len(full_dataset)} rows")
    
    # 4. Train the model
    print("\nStarting model training...")
    trainer, tokenizer = train_model(full_dataset)
    print("Training completed successfully")
    
    # 5. Generate and save predictions
    print("\nGenerating predictions...")
    final_predictions = generate_predictions(trainer, tokenizer, full_dataset)
    final_predictions.to_csv('final_predictions.csv', index=False)
    print("Predictions saved to final_predictions.csv")
    
    # 6. Evaluate trained model performance
    print("\nEvaluating trained model...")
    trained_model_metrics = evaluate_docstrings(final_predictions)
    print("\nTrained Model Performance:")
    print(json.dumps(trained_model_metrics, indent=2))
    
except Exception as e:
    print(f"\nError in pipeline: {str(e)}")
    raise

Found 16 files to process

Evaluating all models...


""



Loading datasets for training...
Loaded deep_chain.csv (5 rows)
Loaded deep_few.csv (5 rows)
Loaded deep_oneshot.csv (5 rows)
Loaded deep_structured.csv (5 rows)
Loaded deep_zero.csv (5 rows)
Loaded generated_docs_.csv (5 rows)
Loaded llama_chain.csv (5 rows)
Loaded llama_few.csv (5 rows)
Loaded llama_oneshot.csv (5 rows)
Loaded llama_structured.csv (5 rows)
Loaded llama_zero.csv (5 rows)
Loaded qwen_chain.csv (5 rows)
Loaded qwen_few.csv (5 rows)
Loaded qwen_oneshot.csv (5 rows)
Loaded qwen_structured.csv (5 rows)
Loaded qwen_zero.csv (5 rows)

Combined dataset contains 80 rows

Starting model training...


Map:   0%|          | 0/75 [00:00<?, ? examples/s]

C:\Users\adenk\PycharmProjects\code-to-doc\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(



Error in pipeline: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

In [66]:
!pip install transformers sentence-transformers rouge-score pandas datasets torch

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for absl-py from https://files.pythonhosted.org/packages/f6/d4/349f7f4bd5ea92dab34f5bb0fe31775ef6c311427a14d5a5b31ecb442341/absl_py-2.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/4d/66/7d9e26593edda06e8cb531874633f7c2372279c3b0f46235539fe546df8b/nltk-3.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/7e/d4/7ebdbd03970677812aac39c869717059dbb71a4cfc033ca6e5221787892c/click-8.1.8-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/135.6 kB ? eta -:--:--
   --- -----


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
